In [14]:
import pandas as pd
import numpy as np
import requests
from io import StringIO

In [15]:
url = "https://www.ncei.noaa.gov/data/global-hourly/access/2024/01087899999.csv"
response = requests.get(url)
data = StringIO(response.text)

df = pd.read_csv(data)

print(df.head())
print(df.dtypes)

      STATION                 DATE  SOURCE  LATITUDE  LONGITUDE  ELEVATION  \
0  1087899999  2024-01-01T00:20:00       4     61.45   2.133333       90.0   
1  1087899999  2024-01-01T00:50:00       4     61.45   2.133333       90.0   
2  1087899999  2024-01-01T01:20:00       4     61.45   2.133333       90.0   
3  1087899999  2024-01-01T01:50:00       4     61.45   2.133333       90.0   
4  1087899999  2024-01-01T02:20:00       4     61.45   2.133333       90.0   

                        NAME REPORT_TYPE  CALL_SIGN QUALITY_CONTROL  ...  AW2  \
0  SNORRE A OIL PLATFORM, NO       FM-15      99999            V020  ...  NaN   
1  SNORRE A OIL PLATFORM, NO       FM-15      99999            V020  ...  NaN   
2  SNORRE A OIL PLATFORM, NO       FM-15      99999            V020  ...  NaN   
3  SNORRE A OIL PLATFORM, NO       FM-15      99999            V020  ...  NaN   
4  SNORRE A OIL PLATFORM, NO       FM-15      99999            V020  ...  NaN   

                  GA1                 GA2   

In [16]:
df['DATE'] = pd.to_datetime(df['DATE'])

df['TMP'] = df['TMP'].astype(str)
df['SLP'] = df['SLP'].astype(str)
df['DEW'] = df['DEW'].astype(str)

df['TMP'] = df['TMP'].str.slice(0, -2)
df['SLP'] = df['SLP'].str.slice(0, -2)
df['DEW'] = df['DEW'].str.slice(0, -2)

df['TMP'] = pd.to_numeric(df['TMP'], errors='coerce')
df['SLP'] = pd.to_numeric(df['SLP'], errors='coerce')
df['DEW'] = pd.to_numeric(df['DEW'], errors='coerce')
df.dropna(subset=['TMP', 'DEW', 'SLP'], inplace=True)

df.drop_duplicates(inplace=True)

In [17]:
df['Temp_Pressure_Diff'] = df['TMP'] - df['SLP']

df['MONTH'] = df['DATE'].dt.month

monthly_agg = df.groupby('MONTH').agg(Average_Temp_Dew_Diff=('TMP', lambda x: (x - df.loc[x.index, 'DEW']).mean())).reset_index()

print(monthly_agg)

   MONTH  Average_Temp_Dew_Diff
0      1              41.610284
1      2              45.592727
2      3              45.481481
3      4              41.421170
4      5              38.735067
5      6              35.345912
6      7              27.842179
7      8              29.788136
8      9              31.057692
